# Criptografía - Práctica 1: Aritmética modular
Guillermo Chica Sabariego

### Ejercicio 1: Implementación del algoritmo extendido de Euclides para el cálculo del máximo común divisor

In [2]:
def mcd(a,b):
    """
    Implementación del algoritmo de euclides para el cálculo del MCD y los
    coeficientes de Bezout
    
    Entrada: 
        a, b: enteros 
    
    Salida:
        El mcd y los coeficientes u y v
    """


    #Inicilizamos u y v para a y b
    u_a=1
    v_a=0
    u_b=0
    v_b=1

    while(b!=0):
        #Hallamos cociente
        c = a / b
        #Valores futuros de u y v
        aux_u = u_a - c*u_b
        aux_v = v_a - c*v_b
        #Valor futuro d b
        aux_b = a - c*b
        #Hacemos los cambios
        u_a=u_b
        v_a=v_b
        u_b=aux_u
        v_b=aux_v
        a=b
        b=aux_b
        
    #a es el mcd, u el inverso si a es 1
    return [a,u_a, v_a]

Al final del algoritmo obtenemos no solo el mcd, guardado en el valor de `a` sino también los coeficientes de Bézout:
*d = au + bv* (*d* es el mcd). En nuestro caso, `u_a` representa el coeficiente *u* y `v_a` el coeficiente *v*.

Algunas pruebas:

In [4]:
mcd(48,60)

[12, -1, 1]

In [9]:
48*-1+60*1 #d = au + bv

12

In [5]:
mcd(42,56)

[14, -1, 1]

In [10]:
42*-1+56*1

14

In [8]:
mcd(1468,964)

[4, 44, -67]

In [11]:
1468*44+964*-67

4

Se puede comprobar en todos los casos que se cumple la identidad de Bézout, *d = au + bv*

### Ejercicio 2: Inverso modular

In [3]:
def inverso(a,n):
    """
    Algortimo para calcular el inverso modular de un número
    
    Entrada: 
        a y n enteros
    
    Salida: 
        El inverso modulo n de a: a^-1 mod n
    """
    d_u=mcd(a,n)
    if (d_u[0]!=1):
        print "No existe el inverso modulo "+str(n)+ " para "+str(a)
    else:
        #1=u*a+v*n -> u es el inverso
        return d_u[1]%n

Sabemos que un número a tiene inverso modulo n si el máximo común divisor de a y n vale 1 (son primos relativos). En ese caso, la identidad de Bézout quedaría así:

*1 = au + nv*

Pero *nv* vale cero porque cualquier número multiplicado por *n mod n* da cero.

*1 = au*

*1/a = u*

Luego u es el inverso de a módulo n.

In [14]:
inverso(46381,59)

17

In [15]:
(46381*17) % 59

1

Efectivamente, cumple que un número multiplicado por su inverso vale 1.

### Ejercicio 3: Exponenciación modular

Escribimos un función que calcule *a <sup>b</sup> mod n* para cualesquiera *a*, *b* y *n* enteros positivos, teniendo en cuenta la representación binaria de *b*.

In [4]:
def expmod(b,e,m):
    """
    Calcula la potencia modular de un número
    
    Entrada:
        b: base entera
        
        e: exponente entero
       
        m: modulo entero
    
    Salida:
        prod: resultado de (b^e) mod n
    """
    #b = base
    #e = exponente
    #m = modulo
    s=e
    prod=1 #En prod voy almacenando el resultado de las potencias
    a=b
    
    while(s>0):
        #Caso 1: primer bit es 1
        if(s%2==1):
            #Cambio prod
            prod = (prod * a) % m
            #Cambio s
            s = (s-1)/2
        #Caso 2: primer bit es 0
        elif (s%2==0):
            #No cambio prod
            #Cambio s
            s = s / 2
        #Cambio a
        a = (a*a) % m
        if(a==1):
            return prod
    
    #Devuelvo el resultado:
    return prod

A pesar de que no trabajamos con bits, si no con enteros, estamos teniendo en cuenta igualmente la representación binaria de b.

Si el resto de dividir *s / 2* vale *1*, el primer bit vale 1.

Ejemplos:

*5 % 2 = 1* y la representación binaria de *5* es `101`

*61 % 2 = 1* y la representación binaria de *61* es `111101`

Y al contrario, si el resto de dividir *s / 2* vale 0, el primer bit vale 0.

*10 % 2 = 0* y 10 en binario es `01010`

In [21]:
expmod(562390847,1894083629,65398261921)

30005440114L

In [22]:
expmod(32448648,68347136287,13203)

1053

### Ejercicio 4: Test de primalidad con Miller-Rabin

Vamos a escribir una función que dado un candidato a primo p, nos diga si es probablemente primo.

Primero implementamos el algoritmo de Miller-Rabin. El siguiente test, además del teorema pequeño de Fermat hace uso de que si n es primo, entonces en Z<sub>n</sub> un polinomio de grado 2 tiene a lo sumo dos raíces:

In [5]:
from random import randrange

def millerrabin(p):
    """
    Implementación del algoritmo de mille rabin para comprobar si un número
    es primo
    
    Entrada:
        p: número candidato
        
    Salida:
        True: Si es probablemente primo
        
        False: Si no es primo
    """
    
    #Expresamos p-1 como (2**u)*s
    #Calculamos s y u
    s = p-1
    u=0
    while (s%2==0): #vemos si es primo
        s = s//2
        u=u+1
    
    
    #Escogemos un a aleatorio entre 2 y p-2 Unif[2,p-2] p-2>2
    #randrange(inicio, fin, salto)
    #inicio: se incluye en el rango
    #fin: fin del rango, no se incluye en el rango
    a = randrange(2,p-1) 
       
    #Si a**s=1 o a**s=-1
    if(expmod(a,s,p)==1 or expmod(a,s,p)==p-1): #p-1 es -1 en mod p
        return True
    else:
        #Desde i=1 hasta u-1
        for i in range(1,u, 1):
            #Si a^((2^k)*u) igual a -1 con k<u -> probablemente primo
            exp=(2**i)*s
            if (expmod(a,exp,p)==p-1): 
                return True
            #Si a^((2^k)*u) igual a 1 no precedido de -1 -> no primo
            elif (expmod(a,exp,p)==1):
                return False
        #Si no aparece el 1-> No primo
        return False

In [26]:
millerrabin(13)

True

Sin embargo, miller-rabin depende de un número aleatorio *a* (una base) para determinar si un número es probablemente primo o no, así que dependiendo del valor de a podemos obtener unas veces que no es primo y otras que probablemente primo, para un mismo candidato. Si un número n es compuesto, el número de bases para las que pasa el test de Miller-Rabin es, como mucho, *(n−1)/4*. Por tanto, la probabilidad de que el test falle para un número compuesto
es menor o igual que 1/4. Por ejemplo:

In [27]:
millerrabin(52)

True

In [29]:
millerrabin(52)

False

Para subsanar este problema, hacemos una función que pase el algoritmo de miller-rabin a un número varias veces. Si repetimos el test para m bases distintas, y el número que probamos es compuesto, la probabilidad de que falle para todos es menor o igual que 1/4<sup>m</sup>. Por tanto, si tomamos un número, y le pasamos el test m veces, y en todas las ocasiones nos dice que podría ser primo, la probabilidad de que lo sea es mayor o igual que *(4m-1)/4m*:

In [6]:
def esprimo(p,n):
    """
    Función que pasa el test de miller rabin varias veces, y dice si es primo
    o no con un porcentaje
    
    Entrada: 
        p: número que pasa test
        
        n: número de veces que se hace el test
    
    Salida: 
        True si primo, False si no primo
    """
    siprimo=0   
    for i in range (1,n,1):
        primo = millerrabin(p)
        if (primo==True):
            siprimo=siprimo+1
        else:
            return False
    #Calculamos porcentajes:
        
    porcentajeprimo = (float(siprimo)/n)*100
    
    #Además, calculamos la probabilidad de que sea primo
    prob = ((4**n-1)/float(4**n))*100
    
    print "Probablemente primo. El "+str(porcentajeprimo)+" % de veces ha dado probablemente primo"
    print "Es primo con una probabilidad mayor o igual que "+str(prob)+" %"
    return True

In [51]:
esprimo(577,10)

Probablemente primo. El 90.0 % de veces ha dado probablemente primo
Es primo con una probabilidad mayor o igual que 99.9999046326 %


True

In [52]:
esprimo(123456789101119,10)

Probablemente primo. El 90.0 % de veces ha dado probablemente primo
Es primo con una probabilidad mayor o igual que 99.9999046326 %


True

In [53]:
esprimo(9972,10)

False

In [55]:
esprimo(28564333765949,20)

Probablemente primo. El 95.0 % de veces ha dado probablemente primo
Es primo con una probabilidad mayor o igual que 99.9999999999 %


True

### Ejercicio 5: Algortimo Paso enano-paso gigante para el cálculo de logartimos discretos en Z<sub>p</sub>

Suponiendo *p* un número primo, *a*, *b*, dos enteros no nulos, el algorimo paso enano paso gigante calcula *x* tal que *a<sup>x</sup> = b mod p*

In [7]:
def petitpasgrandpas(a,b,p):
    """
    Implementación del algoritmo paso enano-paso gigante
    Calcula x tal que a^x = b mod p
    
    Entrada: 
        a, b: enteros
        
        p: primo
    
    Salida: 
        x tal que a^x= b mod p si hay solución 
        
        False si no hay solución
    
    """
    #Calculamos s: primer primo que elevado a 2 da > que p-1
    s=1
    while(s**2<p-1):
        s=s+1
    
    #Sacamos la lista grande y la pequeña
    L = [expmod(a,s*i,p) for i in range(1,s+1)]
    l = [(b*expmod(a,i,p))%p for i in range(s)]
    
    #Buscamos la insterseccion entre L y l
    n = list(set(L) & set(l))
    
    #Si no hay intersección, no hay solución
    if not n:
        print "No hay solución"
        return False
    
    #x = i*b - j
    #i indice de n en L +1
    #j indice de n en l
    i = L.index(n[0]) + 1
    j = l.index(n[0])
    
    x = i*s -j
    
    return x #solución del logaritmo discreto

In [7]:
petitpasgrandpas(364879542899,8590365927553,562390847)

240307345

### Ejercicio 6: Raíces cuadradas modulares

 Sea *n = pq* con *p* y *q* enteros primos positivos:

Escribimos una función que, dado un entero *a* y un primo *p* con  *(a/p) = 1* (símbolo de Jacobi), devuelve *r* tal que *r<sup>2</sup> = a mod p*.


Para ello implementamos primero el símbolo de Jacobi:

In [8]:
def jacobi(a,p):
    """
    Extensión del simbolo de legendre para p no primo

    Entrada:
        a: entero
        p: impar
        
    Salida:
        1 si a es residuo cuadrático mod p
        
        0 si p divide a a
        
        -1 en caso contrario
        
    """   
    if(p!=1): #Expresamos en módulo p para simplificar
       a = a%p
    else: #si (a/1)
        return 1
    
    #Comprobamos los casos base
    if (a==0):
        return 0
    if (a==1):
        return 1
    if (a==2):
        if ( p%8 == 1 or p%8 == 7):
            return 1
        else:
            return -1
    if (a == p-1):
        if (p%4 == 1):
            return 1
        if (p%4 == 3):
            return -1
    
    #saco u y s
    s = a
    u=0
    while (s%2==0): #vemos si es par
        s = s//2
        u=u+1
        
    #Reciprocidad cuadrática
    signo1=1
    signo2=1
    if (u%2==0):
        signo1 = 1
    else:
        if (p%8 == 1 or p%8 == 7):
            signo1 = 1
        else:
            signo1 = -1
    
    if (p%4==1 or s%4==1):
        signo2 = 1
    elif (p%4==3 and s%4==3):
        signo2 = -1
    #Damos la vuelta        
    a = p
    p = s
    
    return signo1*signo2*jacobi(a,p)

El algoritmo comprueba primero los casos base y si de primeras no los cumple, descompone *a* como *2<sup>u</sup>s*, con s impar, lo que se conoce como "sacar doses". Tras eso aplica reciprocidad cuadrática si se puede para dar la vuelta y poner abajo la *s* impar. Así, se repite recursivamente el algoritmo con los nuevos *a* y *p* resultados de dar la vuelta por reciprocidad y se comprueban de nuevo casos base, se sacan doses, etc.

Hay que tener en cuenta que la reciprocidad cuadrática no funciona cuando a y p son los dos impares.

In [18]:
jacobi(15,17)

1

In [19]:
jacobi(7,11)

-1

In [20]:
jacobi(2487548574,1387824574985)

-1

Ahora podemos implementar el algoritmo de raíz cuadrada modular:

In [9]:
def raizmodulo(a,p):
    """
    Dado a y p primo con Jacobi(a,p)=1, devuelve r tal que r²=a mod p
    
    Entrada:
        a: entero
        
        p: primo
    
    Salida:
        r: raiz de a mod p si a es un residuo cuadrático modulo p
        
        False: si a no es un residuo cuadrático modulo p
    """
    
    #Comprobamos que a es un residuo cuadrático mod p
    if(jacobi(a,p)!=1):
        return False
    
    #Expresamos p-1 como (2**u)*s
    #Calculamos s y u
    s = p-1
    u=0
    while (s%2==0):
        s = s//2
        u=u+1
    
    if (u==1):
        r = a**((p+1)/4)
        return r
    if (u>=2):
        #Busco n tal que jacobi(n,p)=-1
        n=1
        while (jacobi(n,p)!=-1):
            n = n+1
        #b es n**s mod p
        b = expmod(n,s,p)
        r = expmod(a, (s+1)/2, p)
        inversoa = inverso(a,p)
        
        for j in range(u-1):
            
            base = expmod(r,2,p) * inversoa
            exponente = 2**(u-2-j)
            if (expmod(base,exponente,p)) == p-1:
                r = (r*b)%p
            
            b = expmod(b,2,p)
        
        return r

Aunque solo devuelvo un valor, el resultado en realidad son dos, *r* y su inverso *p-r*

Algunas comprobaciones:

In [25]:
raizmodulo(2,41)

17

In [26]:
raizmodulo(342545235,28564333765949)

False

Ahora con *a* un entero que es residuo cuadrático módulo *p* y *q*, usaremos el teorema chino de los restos para calcular
todas las raíces cuadradas de *a mod n* a partir de las raíces cuadradas de *a módulo p* y *q*. Para ello implementamos primero el teorema chino de los restos:

In [10]:
def teoremachino(a,b,p,q):
    """
    Implementación del teorema chino de los restos.
    
    Dado a, b, p, q, devuelve x tal que x=amodp y x=bmodq
    
    Entrada:
        a y b: enteros
        
        p y que: primos
    
    Salida:
        x: solución de x=amodp y x=bmodp. x único modulo pq
    """
    x = inverso(p,q)*(b-a)*p+a
    
    return x%(p*q)

El resultado viene de expresar *x = tp + a* (1) y sustituirlo abajo:
*tp + a = b mod q*

*t = (b - a)p<sup>-1</sup>modq*, donde *modq = sq* con *s* entero

*t = p<sup>-1</sup>(b-a) + sq*

Sustituyo *t* en (1):

*x = p<sup>-1</sup>(b-a)p + sqp + a* en *Z<sub>p</sub>*, luego *sqp = 0*

Ahora, calculando las raíces de *a mod p y q* puedo levantar por el teorema chino de los restos y calcular las de *a mod n*:

In [36]:
def raizmodn(a,p,q):
    """
    Algoritmo que calcula las raices cuadradas de a mod n a partir de las
    raices de a mod p y q, con n=pq
    
    Entrada: 
        a entero residuo cuadrático modulo p y q
        
        p y q: primos
    
    Salida: 
        raices de a mod n con n=p*q
        
        False si a no es residuo cuadrático modulo p y q
    """
    if (jacobi(a,p)!=1 and jacobi(a,q)!=1):
        return False
    
    n = p*q
    
    r1 = raizmodulo(a,p)
    r2 = raizmodulo(a,q)
    
    s1 = teoremachino(r1,r2,p,q) #caso r1 r2
    s2 = n - s1 #caso -r1 -r2
    s3 = teoremachino(r1,q-r2,p,q) #caso r1 -r2
    s4 = n - s3 #caso -r1 r2
    
    return [s1,s2,s3,s4]

Tras comprobar que a es residuo cuadrático calculo las raíces de a modulo *p* y *q*. Después aplico el teorema chino para r1 y r2 y obtengo una raíz. De ella saco su inversa sabiendo que es *n-s1*. Podría obtenerla igualmente haciendo el teorema chino de -r1 y -r2, pero así es más rápido. Lo mismo para el resto de casos.

In [10]:
raizmodn(2,7,41)

[263, 24, 270, 17]

In [31]:
raizmodn(46381,8590365927553,28564333765949)

False

### Ejercicio 7: Factorización de enteros

Implementación del método de Fermat para factorización de enteros. Implementamos antes el método de Newton para el cálculo de raíces cuadradas, que nos servirá en el método de Fermat:


In [12]:
def isqrt(n):

    """
    Implementación del método de newton para el cálculo de raíces cuadradas
    
    Entrada: 
        n: entero
    
    Salida:
        x: raíz cuadrada de n
    """    
    if(n==0):
        return 0
    if(n==1):
        return 1

    x = n

    y = (x + 1) // 2
    while y < x:
        x = y
        y = (x + n // x) // 2
    return x

Método de Fermat, se basa en la representación de un número natural impar como diferencia de cuadrados:

In [13]:
def factoriza(n):
    """
    Implementación del método de Fermat para factorización de enteros
    
    Entrada:
        n: entero positivo
        
    Salida:
        a y b tal que n=a*b no triviales (ni 1 ni -1)
        False si no consigue factorizar porque no existe un a y b equidistantes (ejemplo 14)
    """
    #Comprobamos que n no es primo
    if(esprimo(n,10)):
        return False
    #x raiz de n redondeando por arriba
    x = isqrt(n) + 1
    
    while(x<n):
        k = x**2 - n
        #Si k es un cuadrado
        if (isqrt(k)**2 == k): #n = (x^2-k) = (x²-l²) = (x-l)*(x+l)
            l = isqrt(k)
            a = x-l
            b = x+l
            return [a,b]
        else:
            x = x+1
    
    return False

In [15]:
factoriza(77)

[7, 11]

In [29]:
factoriza(5959)

[59, 101]

In [22]:
factoriza(13)

Probablemente primo. El 90.0 % de veces ha dado probablemente primo
Es primo con una probabilidad mayor o igual que 99.9999046326 %


False

In [23]:
factoriza(14)

False

En este último caso da False porque aunque 14 no es primo, no se puede descomponer en dos números que sean equidistantes, que es lo que hace el método de Fermat. En el caso de 77 sí funciona porque 7 y 11 están separados dos unidades de 9, son equidistantes, pero 14 es 7x2, que no son equidistantes.

Además el algoritmo tiene la debilidad de que para números pequeños es fácil rápido sacar los factores, porque estarán más cerca entre sí.

Implementación del algortimo de factorización rho de Pollard:

Este algoritmo hace uso de una función pseudoaleatoria:

In [14]:
def f(x,c,n):
    """
    Función pseudoaleatoria usada en el cálculo de la rho de pollard
    
    Entrada:
        x,c,n, enteros
        
    Salida:
        Valor aleatorio
    """

    return (x**2 + c)%n

In [15]:
from random import randint
def pollard(n):
    """
    Implementación de ro de pollard para factorización de enteros
    
    Entrada:
        n: entero NO PRIMO, el numero que se quiere factorizar
    
    Salida:
        d factor no trivial de n
        
        False si n es primo
    """
    #Comprobamos que n no es primo
    if(esprimo(n,10)):
        return False
    
    while(True):
        x = 2
        y = 2
        d = 1
        c = randint(1,n-1) #c != de 0 y -2
        
        while (d==1):
            x = f(x,c,n)
            y = f(f(y,c,n),c,n)
            d = mcd(abs(x-y),n)[0]
        
        if(d !=n): #Si d igual a n, seguimos intentando con otro c
            return d
   
    
    return d

In [28]:
pollard(18446744073709551617)

274177L

### Ejercicio 8: Comparación de tiempos de ejecución

Vamos a comparar los tiempos de ejecución de nuestras funciones implementadas contra las de GAP. Con la función `time()` de python obtenemos el tiempo que tarda el sistema en realizar una función.

In [16]:
import time

La función `time` de python nos devolverá el tiempo en segundos que tarda en realizarse una batería de pruebas.

#### Máximo común divisor:

In [24]:
start = time.time()
for i in range(100000):
    mcd(123456789101119,28564333765949)
    mcd(8590365927553,28564333765949)
    mcd(8590365927553,123456789101119)
print time.time() - start

2.81551790237


Hemos hecho un bucle grande para poder contabilizar un tiempo relativamente grande, ya que hacer solo una prueba de la función nos arrojaría valores que tienden a cero. En GAP hemos repetido la mismo batería, usando la función GcdRepresentation(a,b), que devuelve los coeficientes de la identidad de Bézout, como nuestra función de pyhton.

`gap> for i in [1..100000] do`

`> GcdRepresentation(123456789101119,28564333765949);`

`> GcdRepresentation(8590365927553,28564333765949);`

`> GcdRepresentation(8590365927553,123456789101119);`

`> od;time;`

`4464`

Nos ha devuelto 4.464 segundos, tardando casi dos segundos más que nuestra función

El caso de la función inverso no lo hacemos porque usa el mismo algoritmo que el mcd.

#### Exponenciación modular:

In [30]:
start = time.time()
for i in range(1,100000):
    expmod(123456789101119,123456789101119,28564333765949)
print time.time() - start

2.7037768364


En GAP usamos PowerModInt:

`gap> for i in [1..100000] do`

`> PowerModInt(123456789101119,123456789101119,28564333765949);`

`> od;time;`

`1168`

Tarda menos GAP con 1.168 segundos frente a los 2.7 de nuestro algoritmo en python


#### Test de primalidad:

In [33]:
start = time.time()
esprimo(123456789101119,10)
print time.time() - start

Probablemente primo. El 90.0 % de veces ha dado probablemente primo
Es primo con una probabilidad mayor o igual que 99.9999046326 %
0.00197219848633


Obtenemos 1.9 milisegundos haciendo el algoritmo millerrabin 10 veces. En GAP usamos IsPrime:

gap> IsPrime(123456789101119);time;

true

4

Y tarda más GAP, 4 milisegundos.

#### Paso enano paso gigante

In [28]:
start = time.time()
petitpasgrandpas(46381,768479,9476407)
print time.time()-start

0.0777220726013


gap> LogMod(768479,46381,9476407);time;

9

En GAP tardamos 9 milisegundos frente a los 77.7 milisegundos de nuestra implementación. Seguramente la nuestra tarda más porque genera las dos listas completas y luego las compara, en lugar de ir comparando mientras va generando.


### Raíces modulares

Comparamos primero las implementaciones del símbolo de Jacobi:

In [31]:
start = time.time()
for i in range(100):
    jacobi(2487548574,1387824574985)
print time.time()-start

0.00418591499329


gap> for i in [1..100] do
> Jacobi(2487548574,1387824574985);
> od;time;
3

3 milisegundos en GAP frente a 4 milisegundos en nuestra implementación.


Para el caso de las raices modulares:

In [44]:
start = time.time()
for i in range(100):
    raizmodn(2,7,41)
print time.time()-start

0.0121178627014


En GAP obtenemos 11 ms:

gap> for i in [1..100] do

> RootsMod(2,7*41);

> od;time;

11


#### Factorización

In [54]:
start = time.time()
factoriza(327964272)
print time.time() - start

1.27880001068


In [60]:
start = time.time()
print pollard(327964272)
print time.time()-start

3
0.000440120697021


gap> Factors(327964272);time;

[ 2, 2, 2, 2, 3, 17, 401917 ]

0


Ambos tardan del orden de 0 segundos en hacer estas operaciones. El método de Fermat es mucho más lento.